## scikeras
- ! pip install scikeras
- https://www.adriangb.com/scikeras/stable/quickstart.html

In [20]:
import scikeras
from sklearn.datasets import make_classification, load_iris
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, Flatten
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import plot_model, set_random_seed

from sklearn.model_selection import train_test_split

In [22]:
# 데이터 로딩
X, y=load_iris(as_frame=True, return_X_y=True)

## Training a model

In [23]:
# X, y = make_classification(1000, 20, n_informative=10, random_state=0)
# X = X.astype(np.float32)
# y = y.astype(np.int64)

def get_model(hidden_layer_dim, meta):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]

    model = keras.models.Sequential()
    model.add(keras.layers.Dense(n_features_in_, input_shape=X_shape_[1:]))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(hidden_layer_dim))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(n_classes_))
    model.add(keras.layers.Activation("softmax"))
    return model

clf = KerasClassifier(
    get_model,
    loss="sparse_categorical_crossentropy",
    hidden_layer_dim=100,
)

clf.fit(X, y)
y_proba = clf.predict_proba(X)

5/5 [==============================] - 0s 1ms/step


## In an sklearn Pipeline

In [24]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', clf),
])

pipe.fit(X, y)
y_proba = pipe.predict_proba(X)

5/5 [==============================] - 0s 2ms/step


## Grid search

In [25]:
params = {
    "hidden_layer_dim": [50, 100, 200],
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.1],
}
gs = GridSearchCV(clf, params, refit=False, cv=3, scoring='accuracy')

gs.fit(X, y)
print(gs.best_score_, gs.best_params_)

2/2 [==============================] - 0s 3ms/step
0.5133333333333334 {'hidden_layer_dim': 100, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'sgd', 'optimizer__learning_rate': 0.1}
